In [3]:
from dotenv import load_dotenv, find_dotenv
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

### 1. LLM Chain

In [5]:
llm = ChatOpenAI()

In [6]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in a few ways:\n\n1. Automated Testing: Langsmith can be used to automate testing processes, allowing for faster and more efficient testing of software applications.\n\n2. Scripting: Langsmith can be used to create testing scripts that can be easily executed across different platforms and environments.\n\n3. Data Generation: Langsmith can be used to generate test data, making it easier to create realistic testing scenarios and evaluate the performance of software applications.\n\n4. Integration Testing: Langsmith can be used to integrate testing with other tools and frameworks, allowing for comprehensive testing of software applications.\n\nOverall, Langsmith can help streamline the testing process and improve the quality of software applications by providing tools and resources for efficient and effective testing.', response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 15, 'total_tokens': 163}, 'model_name': 'g

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [8]:
chain = prompt | llm 

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith is a powerful tool that can greatly assist with testing by providing various features and functionalities that streamline the testing process. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith supports automated testing, allowing you to write test scripts using its scripting language. This can help you automate the testing of your software, saving time and reducing the chances of human error.\n\n2. **Test Data Generation**: Langsmith can generate test data automatically, which can be useful for testing different scenarios and edge cases without having to manually input the data each time.\n\n3. **Integration Testing**: Langsmith can be used for integration testing, where you can test how different components of your software interact with each other.\n\n4. **Performance Testing**: Langsmith can help with performance testing by allowing you to simulate different load scenarios and analyze the performance of y

In [10]:
output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in several ways:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing of language-related features in software applications. By writing scripts in Langsmith to simulate user interactions involving different languages, testers can quickly and efficiently verify the functionality and performance of multilingual applications.\n\n2. **Localization Testing**: Langsmith can assist in localization testing by generating a wide variety of language-specific data, such as text strings, dates, numbers, and special characters. Testers can use this data to ensure that the application displays and behaves correctly in different languages and locales.\n\n3. **Boundary Testing**: Testers can leverage Langsmith to perform boundary testing with language-specific inputs. By generating edge cases and unusual language combinations, testers can identify potential vulnerabilities or unexpected behaviors in the application.\n\n4. **Regression 

### 2. Retrieval Chain

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [14]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [15]:
embeddings = OpenAIEmbeddings()

In [16]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We can pass documents directly, but here we will use retriever. 

In [18]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help visualize test results.'

In [19]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create test cases on the fly, and export test cases from application traces. It also provides a comparison view to track and diagnose regressions in test scores across multiple revisions of an application, a playground environment for rapid iteration and experimentation, and monitoring charts to track key metrics over time. Additionally, LangSmith supports custom evaluations (LLM and heuristic-based) to score test results and allows for A/B testing changes in prompt, model, or retrieval strategy.


### 3. Conversation Retrieval Chain

Now we will create a chatbot, that based on chat history will retrieve relevant documents, and then continue conversation based on the documents retireved. 

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [22]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [23]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM appli

In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [25]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly u

### 4. Agent

In [31]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [27]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [29]:
search = TavilySearchResults()

In [30]:
tools = [retriever_tool, search]

In [32]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [33]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where i

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith supports testing:\n\n1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, retrieval strategies, and other parameters to understand how the model is performing during the prototyping phase.\n\n2. Debugging: LangSmith provides tracing capabilities that give clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications. LangSmith makes it easy to run custom evaluations to score test results.\n\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an appli

In [34]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 1:28 am PDT Mar 26, 2024. Forecast Valid: 5am PDT Mar 26, 2024-6pm PDT Apr 1, 2024 .'}, {'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detec

{'input': 'what is the weather in SF?',
 'output': 'I found some sources where you can check the weather in San Francisco:\n1. [NOAA National Weather Service](https://forecast.weather.gov/zipcity.php?inputstring=San%20francisco,CA)\n2. [The Weather Channel](https://weather.com/weather/tenday/l/San%20Francisco%20CA%20USCA0987:1:US)\n3. [AccuWeather - Hourly Weather Forecast](https://www.accuweather.com/en/us/san-francisco/94103/hourly-weather-forecast/347629)\n4. [AccuWeather - Current Weather](https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629)\n5. [AccuWeather - Weather Forecast](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629)\n\nYou can visit these websites to get the latest weather conditions and forecasts for San Francisco.'}

In [35]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug 

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith is a platform for LLM application development, monitoring, and testing. Here are some ways LangSmith can help test your LLM applications:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for prototyping LLM applications.\n2. Debugging: LangSmith provides tracing capabilities to help debug issues in LLM applications, making it easier to identify and root-cause problems.\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, allowing for test-driven development.\n4. Comparison View: LangSmith offers a comparison view to track and diagnose regressions in test scores across multiple revisions of your application.\n5. Playground: LangSmith provides a playground environment for rapid 